# Exploratory Data Analysys

In [1]:
import pandas as pd
import numpy as np
import os
import matplotsoccer
import math

In [2]:
# constants

PATH: str = "../data/spadl_format/"
LEAGUES: list[str] = ["England", "Spain", "France", "Italy", "Germany"]

In [3]:
teams_df = pd.read_json("../data/wyscout/teams/teams.json")
teams_df

,city,name,wyId,officialName,area,type
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,Vitoria-Gasteiz,Deportivo Alav\u00e9s,696,Deportivo Alav\u00e9s,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
4,Valencia,Levante,695,Levante UD,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
...,...,...,...,...,...,...
137,Las Rozas,Spain,1598,Spain,"{'name': 'Spain', 'id': 724, 'alpha3code': 'ES...",national
138,Beograd,Serbia,17322,Serbia,"{'name': 'Serbia', 'id': 688, 'alpha3code': 'S...",national
139,Bern,Switzerland,6697,Switzerland,"{'name': 'Switzerland', 'id': 756, 'alpha3code...",national
140,Warszawa,Poland,13869,Poland,"{'name': 'Poland', 'id': 616, 'alpha3code': 'P...",national


In [4]:
df_dict = {}
if not os.path.exists("../data/processed/"):
    os.mkdir("../data/processed")
for league in LEAGUES:
    if os.path.exists(f"../data/processed/{league}.csv"):
        df_dict[league] = pd.read_csv(f"../data/processed/{league}.csv")
        continue
    df = pd.read_csv(f"{PATH}{league}.csv", index_col=0)

    # remove not used columns
    df.drop(["original_event_id", "type_name", "result_id", "bodypart_name", "bodypart_id"], inplace=True, axis=1)

    # create column with discretized positions
    # 0 is defense, 1 is midfield and 2 is attack
    df["position"] = df["start_x"].apply(lambda x: 0 if x <= 35 else 1 if x <= 70 else 2)

    # create column with dists between start and end of play. Most important for passes right now
    def calc_dist(row):
        return math.sqrt(math.pow(row["start_x"] - row["end_x"], 2) + math.pow(row["start_y"] - row["end_y"], 2))
    
    # create column with duration of event
    for i in range(df.shape[0]):
        df.loc[i, "duration"] = df.loc[(i+1)%df.shape[0], "time_seconds"] - df.loc[i, "time_seconds"]

    df["distance"] = df.apply(calc_dist, axis=1)
    df_dict[league] = df
    df.to_csv(f"../data/processed/{league}.csv", index=None)

In [11]:
for league in LEAGUES:
    df=df_dict[league]
    vals = {}
    for team_id in df["team_id"].unique():
        aux_df = df[(df["team_id"] == team_id) & (df["type_id"] == 0) & (df["result_name"] == "success")].describe()
        vals[team_id] = aux_df["distance"].iloc[1]
    vals = {k: v for k, v in sorted(vals.items(), key=lambda item: item[1])}
    print(league)
    for k, v in vals.items():
        team_name = teams_df[teams_df["wyId"] == k]["officialName"].iloc[0]
        print(team_name + ": " + str(v))
    

England
Manchester City FC: 17.208294705045727
Arsenal FC: 17.729418216599584
Manchester United FC: 17.82323168977544
AFC Bournemouth: 18.416690138870038
Tottenham Hotspur FC: 18.49016291527559
Crystal Palace FC: 18.55517231543003
Chelsea FC: 18.682360085097265
Southampton FC: 18.80086498340964
Liverpool FC: 18.808336452956677
West Ham United FC: 18.832440595084144
Watford FC: 18.966950078343636
Newcastle United FC: 18.972262064832034
Swansea City AFC: 19.077093616506826
Leicester City FC: 19.188863825446003
Huddersfield Town FC: 19.214874210487316
Stoke City FC: 19.690287465628373
West Bromwich Albion FC: 19.712070043795737
Everton FC: 19.785332507800135
Brighton & Hove Albion FC: 19.83836518037719
Burnley FC: 20.6337458098742
Spain
Club Atl\u00e9tico de Madrid: 17.58148956397191
FC Barcelona: 17.618442299406397
UD Las Palmas: 17.864495605281746
Real Madrid Club de F\u00fatbol: 17.904624229104698
Sevilla FC: 18.06959657763688
Real Betis Balompi\u00e9: 18.65881369540685
Villarreal Club

In [6]:
df_dict["England"]

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,type_id,action_id,result_name,player_name,position,duration,distance
0,2499719,1,2.758649,1609,25413,51.45,34.68,32.55,14.96,0,0,success,A. Lacazette,1,2.188201,27.314619
1,2499719,1,4.946850,1609,370224,32.55,14.96,53.55,17.00,0,1,success,R. Holding,0,1.595338,21.098853
2,2499719,1,6.542188,1609,3319,53.55,17.00,36.75,19.72,0,2,success,M. Özil,1,1.601207,17.018766
3,2499719,1,8.143395,1609,120339,36.75,19.72,43.05,3.40,0,3,success,Mohamed Elneny,1,2.158971,17.493782
4,2499719,1,10.302366,1609,167145,43.05,3.40,75.60,8.16,0,4,success,Bellerín,1,2.246568,32.896202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483894,2500098,2,2785.702032,1633,20620,55.65,7.48,103.95,19.04,0,1136,success,Pedro Obiang,1,3.530634,49.664108
483895,2500098,2,2789.232666,1633,14703,103.95,19.04,103.95,19.04,1,1137,fail,M. Arnautović,2,1.225276,0.000000
483896,2500098,2,2790.457942,1623,8239,2.10,46.92,0.00,46.24,10,1138,success,L. Baines,0,39.363142,2.207351
483897,2500098,2,2829.821084,1633,70965,105.00,0.00,92.40,36.04,5,1139,success,João Mário,2,4.306084,38.179073


In [9]:
for league in LEAGUES:
    df=df_dict[league]
    vals = {}
    for team_id in df["team_id"].unique():
        aux_df = df[(df["team_id"] == team_id) & (df["type_id"] == 0) & (df["result_name"] == "success")].describe()
        vals[team_id] = aux_df["duration"].iloc[1]
    vals = {k: v for k, v in sorted(vals.items(), key=lambda item: item[1])}
    print(league)
    for k, v in vals.items():
        team_name = teams_df[teams_df["wyId"] == k]["officialName"].iloc[0]
        print(team_name + ": " + str(v))

England
Tottenham Hotspur FC: 1.6378525041846883
AFC Bournemouth: 1.7930905019807573
Brighton & Hove Albion FC: 1.991499973453723
Manchester City FC: 2.0340143939805446
Burnley FC: 2.1015325735489125
Arsenal FC: 2.2259105625667637
Stoke City FC: 2.2456200968822064
Chelsea FC: 2.274329910385452
Liverpool FC: 2.440681431206906
Southampton FC: 2.4967042050273434
Swansea City AFC: 2.5673903908012847
West Ham United FC: 2.5711770674786956
West Bromwich Albion FC: 2.5738686169342135
Watford FC: 2.6765755990203517
Crystal Palace FC: 2.6861945000466423
Newcastle United FC: 2.6923665989275887
Everton FC: 2.8115946104001455
Leicester City FC: 2.828976151683904
Huddersfield Town FC: 2.874371302605393
Manchester United FC: 2.8743951779470693
Spain
Getafe Club de F\u00fatbol: 1.2928867230858918
Reial Club Deportiu Espanyol: 1.39373352478197
Levante UD: 1.8192153886199804
Club Atl\u00e9tico de Madrid: 1.8631654020762756
CD Legan\u00e9s: 2.1128631979654906
Real Club Celta de Vigo: 2.113092668496782
F